<a href="https://colab.research.google.com/github/MalihaUCF/Machine-Learning-Course-Assignments-Spring-2019/blob/master/Assignment%202/k_fold_CNN_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#Loading CIFAR-10 dataset

from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import numpy as np
import keras


#creating train,validate and test sets
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images1, val_images1, train_labels1, val_labels1 = train_test_split(train_images,train_labels,test_size=0.0002,stratify=train_labels)

print(train_images.shape)
print(train_images1.shape)
print(val_images1.shape)
print(test_images.shape)
print(train_labels1.shape)


for i in range(0,50):
  print(train_labels[i])

def k_fold_dataset(i,train_images1,train_labels1):
  val_index_a=i*10000
  val_index_b=val_index_a+10000
  
  #splitting train images into train and val set
  #val set
  X_val=train_images1[val_index_a:val_index_b]
  y_val=train_labels1[val_index_a:val_index_b]
  
  #train set
  X_train=np.append(train_images1[0:val_index_a],train_images1[val_index_b:50000],axis=0)
  y_train=np.append(train_labels1[0:val_index_a],train_labels1[val_index_b:50000],axis=0)
  
  
  return X_train,y_train,X_val,y_val
  


(50000, 32, 32, 3)
(49990, 32, 32, 3)
(10, 32, 32, 3)
(10000, 32, 32, 3)
(49990, 1)
[6]
[9]
[9]
[4]
[1]
[1]
[2]
[7]
[8]
[3]
[4]
[7]
[7]
[2]
[9]
[9]
[9]
[3]
[2]
[6]
[4]
[3]
[6]
[6]
[2]
[6]
[3]
[5]
[4]
[0]
[0]
[9]
[1]
[3]
[4]
[0]
[3]
[7]
[3]
[3]
[5]
[2]
[2]
[7]
[1]
[1]
[1]
[2]
[2]
[0]


In [0]:
def get_train_val(n,data,labels):#k=5
    val_start=n*10000
    val_end=val_start+10000
    x_val=data[val_start:val_end]
    y_val=labels[val_start:val_end]
    x_train=np.append(data[0:val_start],data[val_end:50000],axis=0)
    y_train=np.append(labels[0:val_start],labels[val_end:50000],axis=0)
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_val = keras.utils.to_categorical(y_val, num_classes)

    x_train = x_train.astype('float32')
    x_val=x_val.astype('float32')
    return x_train,y_train,x_val,y_val

In [0]:
for i in range(0,5):
    x_train,y_train,x_val,y_val=get_train_val(i,x_main, y_main)
    train_model(i,x_train,y_train,x_val,y_val,x_test, y_test)

In [0]:
batch_size = 32
epochs = 30
data_augmentation = False
num_predictions = 20

from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


#using our best model with k-fold
def train_model1(i,X_train,y_train,X_val,y_val,test_images,test_labels):
  

  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=X_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
 

  #using ADAM

  Adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


  # Let's train our best model using Adam
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam,
                metrics=['accuracy'])



  if not data_augmentation:
      print('Not using data augmentation.')
      history=model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_val, y_val),
                shuffle=True)
  else:
      print('Using real-time data augmentation.')
      # This will do preprocessing and realtime data augmentation:
      datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          zca_epsilon=1e-06,  # epsilon for ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          # randomly shift images horizontally (fraction of total width)
          width_shift_range=0.1,
          # randomly shift images vertically (fraction of total height)
          height_shift_range=0.1,
          shear_range=0.,  # set range for random shear
          zoom_range=0.,  # set range for random zoom
          channel_shift_range=0.,  # set range for random channel shifts
          # set mode for filling points outside the input boundaries
          fill_mode='nearest',
          cval=0.,  # value used for fill_mode = "constant"
          horizontal_flip=True,  # randomly flip images
          vertical_flip=False,  # randomly flip images
          # set rescaling factor (applied before any other transformation)
          rescale=None,
          # set function that will be applied on each input
          preprocessing_function=None,
          # image data format, either "channels_first" or "channels_last"
          data_format=None,
          # fraction of images reserved for validation (strictly between 0 and 1)
          validation_split=0.0)

      # Compute quantities required for feature-wise normalization
      # (std, mean, and principal components if ZCA whitening is applied).
      datagen.fit(X_train)

      # Fit the model on the batches generated by datagen.flow().
      model.fit_generator(datagen.flow(X_train, y_train,
                                       batch_size=batch_size),
                          epochs=epochs,steps_per_epoch=100,validation_steps=50,
                          validation_data=(X_val, y_val))

  
  
  from sklearn.metrics import classification_report, confusion_matrix
  import os
  from keras.models import load_model


  y_pred=model.predict(test_images) 
  y_pred = [np.argmax(entry) for entry in y_pred]
  y_true = [np.argmax(entry) for entry in test_labels]

  print('Confusion Matrix\n\n')
  print(confusion_matrix(y_true, y_pred))  

  print(classification_report(y_pred,y_true))

In [0]:
from random import shuffle

num_classes=10
# Convert class vectors to binary class matrices.
test_labels = keras.utils.to_categorical(test_labels, num_classes)

#train_images=np.array(train_images)
#shuffle(train_images)

for a in range(0,5):
    X_train,y_train,X_val,y_val=k_fold_dataset(a,train_images1,train_labels1)
    
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    X_train /= 255
    X_val /= 255
    
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_val=keras.utils.to_categorical(y_val, num_classes)
    
    print(train_images1.shape)
    print(X_train.shape)
    print(X_val.shape)
    print(test_images.shape)  
    print(y_val.shape)
    
    train_model1(a,X_train,y_train,X_val,y_val,test_images, test_labels)


(49990, 32, 32, 3)
(39990, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 10)
Not using data augmentation.
Train on 39990 samples, validate on 10000 samples
Epoch 1/30
39990/39990 [==============================] - 512s 13ms/step - loss: 1.8280 - acc: 0.3276 - val_loss: 1.6210 - val_acc: 0.4159
Epoch 2/30
39990/39990 [==============================] - 512s 13ms/step - loss: 1.4977 - acc: 0.4568 - val_loss: 1.3925 - val_acc: 0.5051
Epoch 3/30
39990/39990 [==============================] - 516s 13ms/step - loss: 1.3776 - acc: 0.5026 - val_loss: 1.2736 - val_acc: 0.5450
Epoch 4/30
39990/39990 [==============================] - 512s 13ms/step - loss: 1.2871 - acc: 0.5379 - val_loss: 1.1936 - val_acc: 0.5747
Epoch 5/30
39990/39990 [==============================] - 509s 13ms/step - loss: 1.2093 - acc: 0.5701 - val_loss: 1.1202 - val_acc: 0.6017
Epoch 6/30
39990/39990 [==============================] - 511s 13ms/step - loss: 1.1519 - acc: 0.5937 - val_loss: 1.0940 - val_acc: 0.6083

In [0]:
#Saving the model as an h5 file

good_model='adam_dropout_hold_out.h5'
model.save(good_model)

In [0]:
!ls

adam_dropout_hold_out.h5  sample_data


In [0]:
from google.colab import files

with open(good_model, 'r') as f:
  files.download(good_model)